# Конкурентное, параллельное и асинхронное  программирование

## 1. Конкурентное != параллельное

Мы говорим о многозадачности, то есть о выполнении нескольких задач в один и тот же период времени. С многозадачностью связывают конкурентное, параллельное и асинхронное выполнение задач.

**Конкурентные задачи** (англ. concurrent tasks) — задачи, выполнение которых пересекается во времени. Задачи могут выполняться параллельно, а могут порционно (виртуальный параллелизм).

**Параллельные задачи** (англ. parallel tasks) — задачи, которые буквально выполняются одновременно.

Из определений видно, что конкурентность и параллелизм различаются. Это две независимые характеристики, которые образуют 4 вида многозадачности.
![Concurrency/parallelism types](img/parallel_concurrent_table.png)

На рисунке ниже показано различие между последовательным, конкурентным и параллельным выполнением задач. Круги — этапы вычислений, а стрелки показывают последовательность вычислений.

![Concurrency vs parallelism vs sequantial](img/parallel_sequential_concurrent.jpg)

**Асинхронные задачи** (англ. asynchronous tasks) — задачи, которые запущены без ожидания результата. Таким образом, асинхронная задача не блокирует запускающую систему. Обычно асинхронные задачи по окончанию вызывают функцию обратного вызова (англ. callback function).

В противоположность асинхронным задачам ставят синхронные. На рисунке ниже показано время синхронного и асинхронного выполнения задач.

![Sync/async types](img/synchronous-asynchronous.png)

## 2. Модели конкурентного программирования 

Организовать выполение задач конкурентно можно двумя сопособами:
- решение с общей памятью (англ. shared memory),
- решение с передачей сообщений (англ. message passing).

Оба подхода могут быть использованы на машинах с разной архитектурой. Но если задачи выполняются на разных машине с физически разделеённой памятью, то проще огранизовать конкурентую работу на передаче сообщений.

### 2.1. Подход с общей памятью

В этом подходе конкурентные задачи взаимодействуют через общии участки памяти. Такая система может быть построена на разных сущностях: processes, threads, green threads, fibers and coroutines. Описание этих систем приведено ниже.

При этом доступ к общему ресурсу надо синхронизовать. Для этого используются разные механизмы синхоронизации. О них также будет сказано ниже.

#### Processes

#### Threads

#### Green Threads

#### Fibers или Coroutines

#### Способы синхронизации

- Семафоры
- Мьютекс (англ. mutex = mutual exclusion) aka Lock.
- Критические секции


### 2.2. Подход с передачей сообщений

Модель передачи сообщений (англ. Message Passing Model MPI).

- Асинхронная передача сообщений. Actor model
- Синхронная передача сообщений. Communicating sequential processes

### Почитать
- [Такие удивительные семафоры](https://habr.com/en/post/261273/)

## 2. Python threads

Поток (thread) — подзадача процесса операционной системы. Потоки одного процесса делят между собой его общую память. В Python потоки являются системными объtктами, то есть ими управляет операционная система.

In [1]:
from threading import Thread

In [22]:
def subproc(n: int):
    [print(i) for i in range(n)]
    
thread1 = Thread(target=subproc, args=(5,))
thread2 = Thread(target=subproc, args=(5,))

thread1.start()
thread2.start()
thread1.join()
thread2.join()

0
1
2
3
4
0
1
2
3
4


In [40]:
class MyThread(Thread):
    def __init__(self, n:int):
        super().__init__(name=f"Up to {n}")
        self.n = n
    def run(self):
        [print(i) for i in range(self.n)]
        
thread1 = MyThread(5)
thread2 = MyThread(5)

thread1.start()
thread2.start()
thread1.join()
thread2.join()

0
1
2
3
4
0
1
2
3
4


### Демонизация потоков

Процесс будет работать до тех пор, пока все потоки не будут завершены. Если надо по явному завершению процесса прекратить работу всех дочерних потоков, то их надо сделать демонами.

In [31]:
def subproc(n: int):
    [print(i) for i in range(n)]
    
thread1 = Thread(target=subproc, args=(5,), daemon=True)
thread2 = Thread(target=subproc, args=(5,), daemon=True)

thread1.start()
thread2.start()
thread1.join()
thread2.join()

0
1
2
3
4
0
1
2
3
4


### GIL (Global Interpreter Lock)

GIL — глобальный lock интерпретатора. Это особенность реализации Python из-за которой нельзя одновременно использовать несколько процессоров для потоков. Все параллельные вычисления в Python реализуются с помощью модулей.

### Почитать
- [Как устроен GIL в Python](https://habr.com/en/post/84629/)

### Блокировка ресурсов

#### Мьютекс

In [41]:
from threading import Lock

In [47]:
mutex = Lock()

mutex.acquire()
print(mutex)
# Работа с общими ресурсами
mutex.release()
print(mutex)

<locked _thread.lock object at 0x109b52a80>
<unlocked _thread.lock object at 0x109b52a80>


In [64]:
x = 0
def fun_inc(n:int):
    global x
    for _ in range(n):
        x += 1
    
def fun_dec(n:int):
    global x
    for _ in range(n):
        x -= 1

thread1 = Thread(target=fun_inc, args=(5000,))
thread2 = Thread(target=fun_dec, args=(5000,))

thread1.start()
thread2.start()
thread1.join()
thread2.join()

print(x)

0


In [76]:
try:
    mutex.acquire()
    # Работа с общими ресурсами

except:
    # Обрабатываем исключения
    pass

finally:
    mutex.release()

In [ ]:
with

#### Рекурсивный мьютекс

In [74]:
from threading import RLock

#### Семафор

In [67]:
from threading import Semaphore

In [69]:
s = Semaphore(10)

s.acquire()
print(s)
# Работа с общими ресурсами
s.release()
print(s)

#### События

In [70]:
from threading import Event

In [73]:
e = Event()

#e.wait() # Ждём, когда кто-нибудь захватит флаг
print(e)
e.set() # Ставим флаг
print(e)
# Работа с общими ресурсами
e.clear() # Снимаем флаг и ждём нового
print(e)

#### Условия

## 3. Processes

## 4. MPI

## Бонус: аппаратный взгляд на паралелизм

### Классификация Флина

[Классификация Флина](https://en.wikipedia.org/wiki/Flynn%27s_taxonomy)

![Flynn's Taxonomy](img/parallel-architectures.png)

#### Имеет ли смысл распараллеливать задачу? Закон Амдала (англ. Amdahl’s Law)
Закон Амдала показывает во сколько раз меньше времени потребуется параллельной программе для решения задачи по сравнению с последовательной. 
$$
\text{Speed-up}=\frac{1}{S+P/n}
$$